In [1]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from datetime import datetime
import datetime

### Training Model

In [2]:
def pre_train_process(df,startIdx,windowLength,isNormalized):
    tmp = df.copy()
    tmp.reset_index(inplace=True)
    train_start_index = startIdx
    train_window_length = windowLength
    train_end_index = train_start_index+train_window_length-1
    tmp = tmp.iloc[train_start_index:train_end_index]
    ## normalize the option price change and future price change
    if(isNormalized==False):
        tmp['opt_price_chg'] = tmp['opt_price'].diff()
        tmp['fut_price_chg'] = tmp['fut_price'].diff()
        tmp = tmp[tmp['fut_price_chg']!=0]
    else:
        tmp['opt_price_chg'] = tmp['opt_price'] / tmp['opt_price'].shift()
        tmp['fut_price_chg'] = tmp['fut_price'] / tmp['fut_price'].shift()
        tmp = tmp[tmp['fut_price_chg']!=1]
    tmp = tmp[['ts','opt_price_chg','fut_price_chg','delta','vega','days_to_expiry','fut_price']]
    tmp.reset_index(drop=True,inplace=True)
    train_set = pd.DataFrame()
    coeff = tmp['vega']*tmp['fut_price_chg']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    train_set['y'] = (tmp['opt_price_chg'] - tmp['delta']*tmp['fut_price_chg'])
    train_set['x1'] = coeff*tmp['delta']
    train_set['x2'] = coeff*tmp['delta']**2
    train_set.drop(index=0,inplace=True)
    return train_set

In [3]:
def train_model(train_set):
    x = sm.add_constant(train_set.iloc[:,1:])
    y = train_set['y']
    model = sm.OLS(y,x)
    result = model.fit()
    return result

In [4]:
def predict_MV_delta(df,startIdx,windowLength,result):
    tmp = df.copy()
    test_start_index = startIdx
    test_window_length = windowLength
    test_end_index = test_start_index + test_window_length-1
    tmp = tmp.iloc[test_start_index:test_end_index]
    tmp = tmp[['ts','delta','vega','days_to_expiry','fut_price','IV']]
    b0 = result.params[0]
    b1 = result.params[1]
    b2 = result.params[2]
    tmp['y_hat'] = b0+b1*tmp['delta']+b2*tmp['delta']**2
    tmp['MV_delta'] = tmp['delta']+tmp['y_hat']*tmp['vega']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    tmp.reset_index(inplace=True)
    return tmp

In [5]:
def generate_MV_delta(contractName,trainStartIdx,trainLength,isNormalized,predictMVdeltaLength):
    contract = Calls[Calls['opt_symbol']==contractName]
    train_set = pre_train_process(contract,trainStartIdx,trainLength,isNormalized)
    result = train_model(train_set)
    print("result rsquared: ",result.rsquared)
    predict_set = predict_MV_delta(contract,trainStartIdx+trainLength,predictMVdeltaLength,result)
    return predict_set

In [38]:
# generate_MV_delta2 takes option name, predicted date, 
# wheather to normalize data and rsquare threshold to genearte predicted MV_delta
def generate_MV_delta2(contractName,Date,isNormalized,threshold,initNumberOfTrainingData=40):
    contract = Calls[Calls['opt_symbol']==contractName].copy()
    contract.reset_index(inplace=True,drop=True)
    idx = contract.index[contract['ts']<=Date].tolist()[-1]
    #print("available data rows:", idx)
    #i = 200
    i = initNumberOfTrainingData # tuned at 20220212
    rsquare = 0
    while(rsquare<threshold and i <= idx):
        train_set = pre_train_process(contract,idx-i,i,isNormalized)
        result = train_model(train_set)
        rsquare = result.rsquared
        i += 10
    predict_set = predict_MV_delta(contract,idx,2,result)
    if rsquare < threshold:
        #print('WARNING: not enough data to make the regression model stable!')
        predict_set['model stable'] = 0
    else:
        predict_set['model stable'] = 1
    #print('i: ',i)
    #print("result rsquared: ",result.rsquared)
    #print(result.summary())
    return predict_set

In [41]:
def generate_deltas(contractName,startDate,nDays,threshold,initNumberOfTrainingData=40):
    date = startDate
    res = pd.DataFrame(columns=['index','ts','delta','vega','days_to_expiry','fut_price','y_hat','MV_delta'])
    for i in range(0,nDays):
        predicted = generate_MV_delta2(contractName,date.strftime("%Y-%m-%d, %H:%M:%S"),True,threshold,initNumberOfTrainingData=40)
        res = res.append(predicted)
        date = date+datetime.timedelta(1)
    return res

In [8]:
def calculate_PnL(df):
    PnLs = pd.DataFrame(columns=['Delta_PnL','MV_delta_PnL'])
    for index, row in df.iterrows():
        if(index>0):
            delta_PnL = (row['fut_price'] - df.iloc[index-1]['fut_price'])*df.iloc[index-1]['delta']
            MV_delta_PnL = (row['fut_price'] - df.iloc[index-1]['fut_price'])*df.iloc[index-1]['MV_delta']
            df_tmp = pd.DataFrame({'Delta_PnL':[delta_PnL],'MV_delta_PnL':[MV_delta_PnL]})
            PnLs = PnLs.append(df_tmp)
    return PnLs

### Read in data

In [9]:
Calls = pd.read_csv('nifty_311221_190122_1min_calls_py_vollib.csv')
#Calls = Calls[Calls['delta']>=0.05]
#Calls = Calls[Calls['delta']<0.9]
contractList = Calls['opt_symbol'].unique()
#contractList

In [10]:
contractList.tolist()
print(contractList.size)
contractList

79


array(['NIFTY22JAN17450CE', 'NIFTY22JAN16950CE', 'NIFTY22JAN17950CE',
       'NIFTY22JAN18050CE', 'NIFTY22JAN19600CE', 'NIFTY22JAN18650CE',
       'NIFTY22JAN17600CE', 'NIFTY22JAN17300CE', 'NIFTY22JAN17000CE',
       'NIFTY22JAN18150CE', 'NIFTY22JAN16900CE', 'NIFTY22JAN18200CE',
       'NIFTY22JAN18450CE', 'NIFTY22JAN17750CE', 'NIFTY22JAN19000CE',
       'NIFTY22JAN16400CE', 'NIFTY22JAN16700CE', 'NIFTY22JAN18600CE',
       'NIFTY22JAN18100CE', 'NIFTY22JAN16600CE', 'NIFTY22JAN18350CE',
       'NIFTY22JAN17850CE', 'NIFTY22JAN18700CE', 'NIFTY22JAN16500CE',
       'NIFTY22JAN19400CE', 'NIFTY22JAN19450CE', 'NIFTY22JAN17150CE',
       'NIFTY22JAN17550CE', 'NIFTY22JAN19500CE', 'NIFTY22JAN16000CE',
       'NIFTY22JAN18550CE', 'NIFTY22JAN17400CE', 'NIFTY22JAN17050CE',
       'NIFTY22JAN18900CE', 'NIFTY22JAN19050CE', 'NIFTY22JAN19100CE',
       'NIFTY22JAN16300CE', 'NIFTY22JAN17250CE', 'NIFTY22JAN16100CE',
       'NIFTY22JAN18800CE', 'NIFTY22JAN17350CE', 'NIFTY22JAN17700CE',
       'NIFTY22JAN16

In [30]:
contractName = 'NIFTY22JAN19400CE'
#contractName = 'NIFTY22JAN18800CE'
contract = Calls[Calls['opt_symbol']==contractName]
startDate = datetime.datetime(2021,12,31,15,25)
deltas = generate_deltas(contractName,startDate,nDays=7,threshold=0.95,initNumberOfData=40)
deltas = deltas.reset_index()
deltas

,level_0,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta,IV,model stable
0,0,182,2021-12-31 15:29:00,0.014150,350.382276,27.001042,17420.00,-23.055400,-1.402546,0.148321,0.0
1,0,182,2021-12-31 15:29:00,0.014150,350.382276,27.001042,17420.00,-23.055400,-1.402546,0.148321,0.0
2,0,182,2021-12-31 15:29:00,0.014150,350.382276,27.001042,17420.00,-23.055400,-1.402546,0.148321,0.0
3,0,427,2022-01-03 15:29:00,0.014988,339.231104,24.001042,17692.35,3.950790,0.260447,0.136242,0.0
4,0,791,2022-01-04 15:29:00,0.016113,336.822347,23.001042,17856.00,-1.800070,-0.096278,0.127041,0.0
5,0,1157,2022-01-05 15:29:00,0.018772,331.415618,22.001042,17946.20,1.085992,0.086646,0.125634,0.0
6,0,1531,2022-01-06 15:29:00,0.018669,319.296815,21.001042,17795.00,-0.093983,0.012827,0.142271,0.0


In [21]:
pnls = calculate_PnL(deltas)
pnls

,Delta_PnL,MV_delta_PnL
0,8.282405,18.831776
0,6.130567,5.716382
0,-12.443202,-10.114609
0,3.357983,3.658226
0,0.000000,0.000000
0,0.000000,0.000000


In [22]:
print(pnls['Delta_PnL'].sum())
print(pnls['MV_delta_PnL'].sum())

5.327752724413687
18.091774249551243


In [144]:
contractName = 'NIFTY22JAN18750CE'
#contractName = 'NIFTY22JAN18800CE'
contract = Calls[Calls['opt_symbol']==contractName]
contract[contract['ts']<='2022-01-01 14:42:00+05:30'].size

816

### Numerical Experiment

In [11]:
# input of the experiment should be 
# 1. contract name
# 2. test date
# 3. holding period
# 4. treshold

# output include
# 1. contract name
# 2. test date
# 3. holding period
# 4. accumulative PnL delta
# 5. accumulative PnL MV delta
# 6. iv at the start of the holding period
# 7. moneyness
# 8. time to maturity

In [54]:
def SingleNumericalExperiment(contractName, testDate,holdingPeriod,treshold,initNumberOfTrainingData=40):
    contract = Calls[Calls['opt_symbol']==contractName]
    startDate = testDate
    deltas = generate_deltas(contractName=contractName,startDate=testDate,nDays=holdingPeriod+1,threshold=treshold,initNumberOfTrainingData=40)
    deltas = deltas.reset_index()
    pnls = calculate_PnL(deltas)
    res = {'contract name':contractName,'start date':testDate.strftime("%Y-%m-%d, %H:%M:%S"),
           'delta':deltas.iloc[0]['delta'],'MV delta':deltas.iloc[0]['MV_delta'],
           'days_to_expiry':deltas.iloc[0]['days_to_expiry'],'iv':deltas.iloc[0]['IV'],
           'BS delta PnL':pnls['Delta_PnL'].sum(),'MV delta PnL':pnls['MV_delta_PnL'].sum(),
           'holding period':holdingPeriod,'model stable':deltas['model stable'][0]}
    return res

In [53]:
def BatchNumericalExperiment(contractName,startDate,endDate,holdingPeriod = 1,treshold = 0.95,initNumberOfTrainingData=40):
    currentDate = startDate
    result = pd.DataFrame(columns=['contract name','start date','delta','MV delta','days_to_expiry','iv',
                               'BS delta PnL','MV delta PnL','holding period','model stable'])
    while(currentDate<endDate):
        tmp_res = SingleNumericalExperiment(contractName=contractName,testDate=currentDate,
                                            holdingPeriod=holdingPeriod,treshold=treshold,
                                            initNumberOfTrainingData=40)
        result = result.append(tmp_res,ignore_index=True)
        currentDate+=datetime.timedelta(1)
    return result

In [43]:
contractName = 'NIFTY22JAN18500CE'
startDate = datetime.datetime(2022,1,20,15,25)
holdingPeriod = 2
treshold = 0.95
res = SingleNumericalExperiment(contractName=contractName,testDate=startDate,holdingPeriod=holdingPeriod,treshold=treshold,initNumberOfTrainingData=40)
res

{'contract name': 'NIFTY22JAN18500CE',
 'start date': '2022-01-20, 15:25:00',
 'delta': 0.09376696147927784,
 'MV delta': 0.05721562623927297,
 'days_to_expiry': 7.966319444444442,
 'iv': 0.12174134396720658,
 'BS delta PnL': 0.0,
 'MV delta PnL': 0.0,
 'holding period': 1,
 'model stable': 1.0}

In [94]:
d1 = datetime.datetime(2022,1,17,15,25)
d2 = datetime.datetime(2022,1,18,15,25)
print(d1>d2)

False


In [48]:
contract = Calls[Calls['opt_symbol']=='NIFTY22JAN16250CE']
contract

,Unnamed: 0,date,opt_price,volume_x,opt_symbol,fut_price,volume_y,fut_symbol,name,expiry,strike,instrument_type,ts,days_to_expiry,IV,delta,vega
129701,262292,2022-01-07 12:15:00+05:30,1567.75,0.0,NIFTY22JAN16250CE,17812.55,35500.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-07 12:15:00,20.135764,0.156460,0.982047,283.930122
129768,262423,2022-01-07 12:16:00+05:30,1567.75,0.0,NIFTY22JAN16250CE,17805.00,76250.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-07 12:16:00,20.135069,0.180719,0.965200,278.693929
129867,262621,2022-01-07 12:17:00+05:30,1567.75,0.0,NIFTY22JAN16250CE,17805.00,52400.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-07 12:17:00,20.134375,0.180722,0.965200,278.689123
129878,262640,2022-01-07 12:18:00+05:30,1567.75,0.0,NIFTY22JAN16250CE,17817.10,23450.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-07 12:18:00,20.133681,0.121319,0.996553,289.346816
130006,262908,2022-01-07 12:20:00+05:30,1567.75,0.0,NIFTY22JAN16250CE,17813.00,21300.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-07 12:20:00,20.132292,0.154427,0.983240,284.298041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228445,475595,2022-01-13 14:55:00+05:30,2016.65,0.0,NIFTY22JAN16250CE,18250.50,9650.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-13 14:55:00,14.024653,0.274338,0.966095,224.050949
228456,475611,2022-01-13 14:56:00+05:30,2016.65,0.0,NIFTY22JAN16250CE,18252.00,7750.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-13 14:56:00,14.023958,0.269739,0.968390,224.989449
228512,475738,2022-01-13 14:57:00+05:30,2016.65,0.0,NIFTY22JAN16250CE,18257.20,10350.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-13 14:57:00,14.023264,0.250989,0.977070,228.658303
228623,475970,2022-01-13 14:58:00+05:30,2016.65,0.0,NIFTY22JAN16250CE,18253.55,13500.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16250.0,CE,2022-01-13 14:58:00,14.022569,0.264674,0.970850,226.000417


In [50]:
%%time
#contractName = 'NIFTY22JAN19400CE'
#contractName = 'NIFTY22JAN16050CE'
contractName = 'NIFTY22JAN16250CE'
startDate = datetime.datetime(2022,1,10,15,25)
holdingPeriod = 2
treshold = 0.95
endDate = datetime.datetime(2022,1,20,15,25)
result = BatchNumericalExperiment(contractName,startDate,endDate,holdingPeriod = 2,treshold = 0.95,initNumberOfTrainingData=10)
result

Wall time: 631 ms


,contract name,start date,delta,MV delta,days_to_expiry,iv,BS delta PnL,MV delta PnL,holding period,model stable
0,NIFTY22JAN16250CE,"2022-01-10, 15:25:00",0.995064,0.403959,20.022569,0.126524,0.00000,0.000000,1,1.0
1,NIFTY22JAN16250CE,"2022-01-11, 15:25:00",0.995064,0.403959,20.022569,0.126524,0.00000,0.000000,1,1.0
2,NIFTY22JAN16250CE,"2022-01-12, 15:25:00",0.995064,0.403959,20.022569,0.126524,446.98296,181.458417,1,1.0
3,NIFTY22JAN16250CE,"2022-01-13, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
4,NIFTY22JAN16250CE,"2022-01-14, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
5,NIFTY22JAN16250CE,"2022-01-15, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
6,NIFTY22JAN16250CE,"2022-01-16, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
7,NIFTY22JAN16250CE,"2022-01-17, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
8,NIFTY22JAN16250CE,"2022-01-18, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0
9,NIFTY22JAN16250CE,"2022-01-19, 15:25:00",0.971993,0.081384,14.021875,0.262264,0.00000,0.000000,1,1.0


In [26]:
contractList1 = contractList[0:70]
contractList2 = contractList[71:]
contractList2

array(['NIFTY22JAN16050CE', 'NIFTY22JAN16250CE', 'NIFTY22JAN19700CE',
       'NIFTY22JAN19650CE', 'NIFTY22JAN19750CE', 'NIFTY22JAN15850CE',
       'NIFTY22JAN19800CE', 'NIFTY22JAN16350CE'], dtype=object)

In [27]:
%%time
contractList1 = contractList[0:70]
contractList2 = contractList[71:]
contractList2
result = pd.DataFrame(columns=['contract name','start date','delta','days_to_expiry','iv',
                               'BS delta PnL','MV delta PnL','holding period','model stable'])
startDate = datetime.datetime(2021,12,31,15,25)
holdingPeriod = 1
treshold = 0.95
endDate = datetime.datetime(2022,1,20,15,25)
for contractName in contractList1:
    print(contractName)
    contract_res = BatchNumericalExperiment(contractName,startDate,endDate,holdingPeriod,treshold)
    result = result.append(contract_res,ignore_index=True)
result.to_csv('Hedging PnL result.csv')

NIFTY22JAN17450CE
NIFTY22JAN16950CE
NIFTY22JAN17950CE
NIFTY22JAN18050CE
NIFTY22JAN19600CE
NIFTY22JAN18650CE
NIFTY22JAN17600CE
NIFTY22JAN17300CE
NIFTY22JAN17000CE
NIFTY22JAN18150CE
NIFTY22JAN16900CE
NIFTY22JAN18200CE
NIFTY22JAN18450CE
NIFTY22JAN17750CE
NIFTY22JAN19000CE
NIFTY22JAN16400CE
NIFTY22JAN16700CE
NIFTY22JAN18600CE
NIFTY22JAN18100CE
NIFTY22JAN16600CE
NIFTY22JAN18350CE
NIFTY22JAN17850CE
NIFTY22JAN18700CE
NIFTY22JAN16500CE
NIFTY22JAN19400CE
NIFTY22JAN19450CE
NIFTY22JAN17150CE
NIFTY22JAN17550CE
NIFTY22JAN19500CE
NIFTY22JAN16000CE
NIFTY22JAN18550CE
NIFTY22JAN17400CE
NIFTY22JAN17050CE
NIFTY22JAN18900CE
NIFTY22JAN19050CE
NIFTY22JAN19100CE
NIFTY22JAN16300CE
NIFTY22JAN17250CE
NIFTY22JAN16100CE
NIFTY22JAN18800CE
NIFTY22JAN17350CE
NIFTY22JAN17700CE
NIFTY22JAN16800CE
NIFTY22JAN18950CE
NIFTY22JAN18300CE
NIFTY22JAN17900CE
NIFTY22JAN18000CE
NIFTY22JAN17500CE
NIFTY22JAN19200CE
NIFTY22JAN17200CE
NIFTY22JAN17100CE
NIFTY22JAN17800CE
NIFTY22JAN18850CE
NIFTY22JAN17650CE
NIFTY22JAN18250CE
NIFTY22JAN

In [57]:
%%time
contractList1 = contractList[0:70]
contractList2 = contractList[71:]
contractList2
result = pd.DataFrame(columns=['contract name','start date','delta','days_to_expiry','iv',
                               'BS delta PnL','MV delta PnL','holding period','model stable'])
startDate = datetime.datetime(2021,12,31,15,25)
holdingPeriod = 7
treshold = 0.95
endDate = datetime.datetime(2022,1,20,15,25)
for contractName in contractList1:
    print(contractName)
    contract_res = BatchNumericalExperiment(contractName,startDate,endDate,holdingPeriod,treshold)
    result = result.append(contract_res,ignore_index=True)
result.to_csv('Hedging PnL result.csv')

NIFTY22JAN17450CE
NIFTY22JAN16950CE
NIFTY22JAN17950CE
NIFTY22JAN18050CE
NIFTY22JAN19600CE
NIFTY22JAN18650CE
NIFTY22JAN17600CE
NIFTY22JAN17300CE
NIFTY22JAN17000CE
NIFTY22JAN18150CE
NIFTY22JAN16900CE
NIFTY22JAN18200CE
NIFTY22JAN18450CE
NIFTY22JAN17750CE
NIFTY22JAN19000CE
NIFTY22JAN16400CE
NIFTY22JAN16700CE
NIFTY22JAN18600CE
NIFTY22JAN18100CE
NIFTY22JAN16600CE
NIFTY22JAN18350CE
NIFTY22JAN17850CE
NIFTY22JAN18700CE
NIFTY22JAN16500CE
NIFTY22JAN19400CE
NIFTY22JAN19450CE
NIFTY22JAN17150CE
NIFTY22JAN17550CE
NIFTY22JAN19500CE
NIFTY22JAN16000CE
NIFTY22JAN18550CE
NIFTY22JAN17400CE
NIFTY22JAN17050CE
NIFTY22JAN18900CE
NIFTY22JAN19050CE
NIFTY22JAN19100CE
NIFTY22JAN16300CE
NIFTY22JAN17250CE
NIFTY22JAN16100CE
NIFTY22JAN18800CE
NIFTY22JAN17350CE
NIFTY22JAN17700CE
NIFTY22JAN16800CE
NIFTY22JAN18950CE
NIFTY22JAN18300CE
NIFTY22JAN17900CE
NIFTY22JAN18000CE
NIFTY22JAN17500CE
NIFTY22JAN19200CE
NIFTY22JAN17200CE
NIFTY22JAN17100CE
NIFTY22JAN17800CE
NIFTY22JAN18850CE
NIFTY22JAN17650CE
NIFTY22JAN18250CE
NIFTY22JAN